# CSI-6-ARI Week 4 Tutorial — **COMPLETE ANSWERED VERSION**
## Data Preprocessing

This notebook contains **all code cells answered step-by-step**, including the 4 exercises.

---
### Topics covered
1. StandardScaler (standardisation)
2. Categorical encoding (LabelEncoder & OneHotEncoder)
3. Train / Test split
4. End-to-end preprocessing with MinMaxScaler


# Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Setup complete, seed =", SEED)

Setup complete, seed = 42


## Part 1 — Standardisation with `StandardScaler`

**Key formula:** `z = (x - μ) / σ`
After standardisation every feature column has **mean ≈ 0** and **std ≈ 1**.

This is important for distance-based (KNN, SVM, KMeans) and gradient-based models.

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
# --- Option 1: quick one-liner ---
# Feature matrix: 4 samples, 2 features with very different scales
X = np.array([
    [1,  10],
    [3, 100],
    [2,  55],
    [4,  25]
])

# fit_transform: learns mean/std from X and applies z = (x-mean)/std
X_scaled = StandardScaler().fit_transform(X)
print("Standardised X (2 features):")
print(X_scaled)

Standardised X (2 features):
[[-1.34164079 -1.09108945]
 [ 0.4472136   1.52752523]
 [-0.4472136   0.21821789]
 [ 1.34164079 -0.65465367]]


In [4]:
# --- Option 2: using sklearn.preprocessing directly (3 features) ---
import sklearn

X3 = np.array([
    [1,  10,  0],
    [3, 100, 10],
    [2,  55,  5],
    [4,  25,  2]
])

X3_scaled = sklearn.preprocessing.StandardScaler().fit_transform(X3)
print("Standardised X (3 features):")
print(X3_scaled)
# Notice each column is now on the same scale

Standardised X (3 features):
[[-1.34164079 -1.09108945 -1.12832963]
 [ 0.4472136   1.52752523  1.52656362]
 [-0.4472136   0.21821789  0.19911699]
 [ 1.34164079 -0.65465367 -0.59735098]]


---
### Exercise 1 — Check standardisation

**Task:** Create `X2` (shape 5×2) with very different column scales, standardise it, then verify mean≈0 and std≈1.

In [5]:
# Step 1: Create X2 with two features on very different scales
X2 = np.array([
    [1,    10],
    [2,    50],
    [3,   100],
    [4,    25],
    [5,    60],
])
print("Original X2:")
print(X2)

# Step 2: Standardise using StandardScaler
X2_scaled = StandardScaler().fit_transform(X2)
print("\nScaled X2:")
print(X2_scaled)

# Step 3: Compute per-column mean and std
col_means = X2_scaled.mean(axis=0)
col_stds  = X2_scaled.std(axis=0)

print("\nColumn means (should be ~0):", col_means)
print("Column stds  (should be ~1):", col_stds)

# Explanation:
# StandardScaler standardises each column independently.
# Tiny non-zero means (e.g. 1e-16) are just floating-point rounding errors, not actual bias.

Original X2:
[[  1  10]
 [  2  50]
 [  3 100]
 [  4  25]
 [  5  60]]

Scaled X2:
[[-1.41421356 -1.25610542]
 [-0.70710678  0.03220783]
 [ 0.          1.6425994 ]
 [ 0.70710678 -0.77298795]
 [ 1.41421356  0.35428614]]

Column means (should be ~0): [0.00000000e+00 3.33066907e-17]
Column stds  (should be ~1): [1. 1.]


---
## Part 2 — Handling Categorical Variables (Encoding)

Machine learning models need numbers. Two strategies:
- **Label Encoding**: assigns integers (0, 1, 2, …). Implies ordering — OK for ordinal categories.
- **One-Hot Encoding**: creates a binary column per category. No ordering implied — best for nominal categories.

In [6]:
# Original categorical feature matrix
X_cat = np.array([
    ["Red",   "Petrol", "Sedan"],
    ["Black", "Diesel", "Sedan"],
    ["Blue",  "Diesel", "Hatchback"]
])
print(X_cat)

[['Red' 'Petrol' 'Sedan']
 ['Black' 'Diesel' 'Sedan']
 ['Blue' 'Diesel' 'Hatchback']]


In [7]:
from sklearn import preprocessing

# Create a LabelEncoder instance
encoder = preprocessing.LabelEncoder()

# Encode the 'Colour' feature — assigns integers alphabetically
encoded_colours = encoder.fit_transform(["Red", "Black", "Blue"])
print("Label encoded ['Red','Black','Blue']:", encoded_colours)
# Black=0, Blue=1, Red=2  (alphabetical order)

Label encoded ['Red','Black','Blue']: [2 0 1]


In [8]:
# Encoding with a new value 'Yellow'
encoded2 = encoder.fit_transform(["Red", "Black", "Blue", "Yellow", "Red"])
print("Label encoded with Yellow:", encoded2)
# Black=0, Blue=1, Red=2, Yellow=3

Label encoded with Yellow: [2 0 1 3 2]


In [9]:
# Create the DataFrame used in this part
df = pd.DataFrame({
    'Colour':    ['Red', 'Black', 'Blue'],
    'Fuel Type': ['Petrol', 'Diesel', 'Diesel'],
    'Body':      ['Sedan', 'Sedan', 'Hatchback']
})
print(df)

  Colour Fuel Type       Body
0    Red    Petrol      Sedan
1  Black    Diesel      Sedan
2   Blue    Diesel  Hatchback


In [10]:
from sklearn.preprocessing import OneHotEncoder

# Fit OneHotEncoder on the 'Colour' column
ohe = preprocessing.OneHotEncoder()
ohe.fit(df[['Colour']])
print("Unique colour categories:", ohe.categories_)

Unique colour categories: [array(['Black', 'Blue', 'Red'], dtype=object)]


In [11]:
# Transform — produces a sparse matrix; .toarray() makes it dense
ohe_result = ohe.transform(df[['Colour']]).toarray()
print("One-hot encoded colours:")
print(ohe_result)
# Columns order: Black, Blue, Red
# Row 0 (Red)  → [0, 0, 1]
# Row 1 (Black)→ [1, 0, 0]
# Row 2 (Blue) → [0, 1, 0]

One-hot encoded colours:
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


---
###  Exercise 2 — Label encoding vs One-hot encoding

**Task:** Apply both encodings to the `Fuel Type` column, then explain when one-hot is preferred.

In [12]:
# Work on a copy to avoid mutating the original df
df_tmp = df.copy()

# --- Part 1: Label encoding for 'Fuel Type' ---
le = preprocessing.LabelEncoder()
df_tmp["FuelType_LE"] = le.fit_transform(df_tmp["Fuel Type"])
print("DataFrame with Label Encoded Fuel Type:")
print(df_tmp)
print("\nEncoding mapping:", dict(zip(le.classes_, le.transform(le.classes_))))
# Diesel=0, Petrol=1

DataFrame with Label Encoded Fuel Type:
  Colour Fuel Type       Body  FuelType_LE
0    Red    Petrol      Sedan            1
1  Black    Diesel      Sedan            0
2   Blue    Diesel  Hatchback            0

Encoding mapping: {'Diesel': 0, 'Petrol': 1}


In [13]:
# --- Part 2: One-hot encoding for 'Fuel Type' using pd.get_dummies ---
fuel_ohe = pd.get_dummies(df_tmp["Fuel Type"], prefix="Fuel")
print("One-hot encoded Fuel Type:")
print(fuel_ohe)

# Combine with original (drop the original 'Fuel Type' column)
combined = pd.concat([df_tmp.drop(columns=["Fuel Type", "FuelType_LE"]), fuel_ohe], axis=1)
print("\nCombined DataFrame:")
print(combined)

One-hot encoded Fuel Type:
   Fuel_Diesel  Fuel_Petrol
0        False         True
1         True        False
2         True        False

Combined DataFrame:
  Colour       Body  Fuel_Diesel  Fuel_Petrol
0    Red      Sedan        False         True
1  Black      Sedan         True        False
2   Blue  Hatchback         True        False


In [14]:
# --- Part 3: Explanation ---
explanation = """
When to prefer One-Hot Encoding over Label Encoding:

1. One-hot encoding should be used when the categorical variable is NOMINAL
   (no natural ordering), such as fuel type (Diesel, Petrol, Electric).
   Label encoding would incorrectly imply Diesel < Petrol, which has no meaning.

2. Most ML algorithms (logistic regression, SVM, neural networks) treat
   integer-encoded labels as numeric distances, leading to biased predictions.
   One-hot avoids this by treating each category as an independent binary feature.

3. However, if there are MANY unique categories (high cardinality), one-hot
   encoding creates very wide sparse matrices, so label encoding or target
   encoding may be more practical in those cases.
"""
print(explanation)


When to prefer One-Hot Encoding over Label Encoding:

1. One-hot encoding should be used when the categorical variable is NOMINAL
   (no natural ordering), such as fuel type (Diesel, Petrol, Electric).
   Label encoding would incorrectly imply Diesel < Petrol, which has no meaning.

2. Most ML algorithms (logistic regression, SVM, neural networks) treat
   integer-encoded labels as numeric distances, leading to biased predictions.
   One-hot avoids this by treating each category as an independent binary feature.

3. However, if there are MANY unique categories (high cardinality), one-hot
   encoding creates very wide sparse matrices, so label encoding or target
   encoding may be more practical in those cases.



---
## Part 3 — Train / Test Split

`train_test_split` randomly divides data into training and test sets.
- `test_size=0.2` → 80% train, 20% test
- Always set `random_state` for reproducibility

In [15]:
from sklearn.model_selection import train_test_split

# Dataset with 10 samples: 2 numeric features + 1 categorical feature
X = ([12,300,'Red'],[11,280,'Red'],[15,264,'Black'],[9,230,'Blue'],
     [25,459,'Black'],[12,400,'Red'],[42,355,'Blue'],[32,435,'Red'],
     [22,564,'Black'],[21,231,'Red'])

# Target labels
y = [1,2,1,1,1,2,2,1,2,1]

print("Full X:", X)
print("Full y:", y)

Full X: ([12, 300, 'Red'], [11, 280, 'Red'], [15, 264, 'Black'], [9, 230, 'Blue'], [25, 459, 'Black'], [12, 400, 'Red'], [42, 355, 'Blue'], [32, 435, 'Red'], [22, 564, 'Black'], [21, 231, 'Red'])
Full y: [1, 2, 1, 1, 1, 2, 2, 1, 2, 1]


In [16]:
# Split: 80% train, 20% test, fixed seed for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=4
)

print("X_train (8 samples):", X_train)
print("\nX_test  (2 samples):", X_test)
print("\ny_train:", y_train)
print("y_test: ", y_test)

X_train (8 samples): [[25, 459, 'Black'], [21, 231, 'Red'], [15, 264, 'Black'], [42, 355, 'Blue'], [12, 300, 'Red'], [11, 280, 'Red'], [12, 400, 'Red'], [32, 435, 'Red']]

X_test  (2 samples): [[9, 230, 'Blue'], [22, 564, 'Black']]

y_train: [1, 1, 1, 2, 1, 2, 2, 1]
y_test:  [1, 2]


In [17]:
# The test size (20%) → 2 out of 10 samples go to test, 8 to train.
# random_state=4 ensures the same split every time you run the notebook.
print("Train size:", len(X_train), "| Test size:", len(X_test))

Train size: 8 | Test size: 2


---
###  Exercise 3 — Inspect your split

**Task:** Re-split with `test_size=0.3`, print sizes, count 1s and 2s in each subset.

In [18]:
# Step 1: Split with test_size=0.3 (30% test, 70% train)
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X, y, test_size=0.3, random_state=4
)

# Step 2: Print sizes
print("=== Sizes ===")
print(f"X_train: {len(X_train2)} samples")
print(f"X_test:  {len(X_test2)} samples")
print(f"y_train: {len(y_train2)} labels")
print(f"y_test:  {len(y_test2)} labels")

# Step 3: Count class labels
n1_train = y_train2.count(1)
n2_train = y_train2.count(2)
n1_test  = y_test2.count(1)
n2_test  = y_test2.count(2)

print("\n=== Class Counts ===")
print(f"y_train  → class 1: {n1_train}, class 2: {n2_train}")
print(f"y_test   → class 1: {n1_test},  class 2: {n2_test}")

# NOTE: Because the dataset is very small (10 samples), class balance
# can shift noticeably between splits. In practice, use stratify=y
# to preserve the original class proportions in both sets.

=== Sizes ===
X_train: 7 samples
X_test:  3 samples
y_train: 7 labels
y_test:  3 labels

=== Class Counts ===
y_train  → class 1: 4, class 2: 3
y_test   → class 1: 2,  class 2: 1


---
## Part 4 — End-to-End Preprocessing (MinMaxScaler)

**MinMaxScaler** rescales features to a fixed range (default [0, 1]):
`x_scaled = (x - x_min) / (x_max - x_min)`

**⚠️ Key rule — fit on train only:**
Compute `x_min` and `x_max` from `X_train`, then apply to both `X_train` and `X_test`.

In [19]:
# Create the fake_reg.csv dataset (gem stone measurements)
rng = np.random.default_rng(SEED)
n = 1000
feature1 = rng.normal(loc=0.0, scale=1.0, size=n)
feature2 = rng.normal(loc=0.0, scale=1.0, size=n)
noise    = rng.normal(loc=0.0, scale=0.15, size=n)
price    = 50_000 + 12_000*feature1 + 8_000*feature2 + 10_000*noise

fake = pd.DataFrame({"feature1": feature1, "feature2": feature2, "price": price})
fake.to_csv("fake_reg.csv", index=False)
print("Created fake_reg.csv with shape", fake.shape)

Created fake_reg.csv with shape (1000, 3)


In [20]:
# Load the data
df = pd.read_csv('fake_reg.csv')
df.head()

,feature1,feature2,price
0,0.304717,-0.059283,52504.417319
1,-1.039984,-0.729287,30687.078667
2,0.750451,-0.414473,56340.644558
3,0.940565,0.633910,66735.841242
4,-1.951035,0.002993,24504.336744


In [21]:
# Extract features (X) and target (y)
X = df[['feature1', 'feature2']].values
y = df['price'].values

# Split 70% train / 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("X_train shape:", X_train.shape)   # (700, 2)
print("X_test  shape:", X_test.shape)    # (300, 2)
print("y_train shape:", y_train.shape)   # (700,)
print("y_test  shape:", y_test.shape)    # (300,)

X_train shape: (700, 2)
X_test  shape: (300, 2)
y_train shape: (700,)
y_test  shape: (300,)


In [22]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler
scaler = MinMaxScaler()

# CRITICAL: fit ONLY on training data (no data leakage)
scaler.fit(X_train)

# Transform both sets using the training statistics
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

print("X_train after MinMaxScaler (first 3 rows):")
print(X_train[:3])
print("\nX_test after MinMaxScaler (first 3 rows):")
print(X_test[:3])

X_train after MinMaxScaler (first 3 rows):
[[0.41336944 0.65206898]
 [0.50666961 0.39970742]
 [0.54779362 0.53040626]]

X_test after MinMaxScaler (first 3 rows):
[[0.4043664  0.04997697]
 [0.65259223 0.47939214]
 [0.68111567 0.44600036]]


---
###  Exercise 4 — Check for data leakage

**Task:** Verify `X_train` values are in [0,1], check `X_test` range, explain any out-of-range values.

In [23]:
# Step 1: Min and max of X_train after scaling
print("=== X_train (scaled) ===")
print("Per-column min:", X_train.min(axis=0))   # Should both be exactly 0.0
print("Per-column max:", X_train.max(axis=0))   # Should both be exactly 1.0

# Step 2: Min and max of X_test after scaling
print("\n=== X_test (scaled) ===")
print("Per-column min:", X_test.min(axis=0))    # Could be slightly < 0
print("Per-column max:", X_test.max(axis=0))    # Could be slightly > 1

# Step 3: Check if any test values fall outside [0, 1]
outside = ((X_test < 0) | (X_test > 1)).any()
print("\nAny X_test values outside [0, 1]?", outside)

=== X_train (scaled) ===
Per-column min: [0. 0.]
Per-column max: [1. 1.]

=== X_test (scaled) ===
Per-column min: [0.10513995 0.03028872]
Per-column max: [0.96118634 0.98238243]

Any X_test values outside [0, 1]? False


In [24]:
# --- Explanation ---
explanation = """
Why can X_test values fall outside [0, 1]?

1. MinMaxScaler learns x_min and x_max exclusively from X_train.
   It scales values using: (x - train_min) / (train_max - train_min)

2. If X_test contains values BELOW the training minimum, the scaled
   value will be negative (< 0). If X_test values are ABOVE the
   training maximum, the scaled value will exceed 1.

3. This is CORRECT BEHAVIOUR and not a bug. The alternative — fitting
   the scaler on the full dataset — would leak test-set information
   into the preprocessing step, making model evaluation unreliable.
   Slight out-of-range values on test data are acceptable.
"""
print(explanation)


Why can X_test values fall outside [0, 1]?

1. MinMaxScaler learns x_min and x_max exclusively from X_train.
   It scales values using: (x - train_min) / (train_max - train_min)

2. If X_test contains values BELOW the training minimum, the scaled
   value will be negative (< 0). If X_test values are ABOVE the
   training maximum, the scaled value will exceed 1.

3. This is CORRECT BEHAVIOUR and not a bug. The alternative — fitting
   the scaler on the full dataset — would leak test-set information
   into the preprocessing step, making model evaluation unreliable.
   Slight out-of-range values on test data are acceptable.



---
## Summary — Key takeaways from Week 4

| Concept | Tool | When to use |
|---|---|---|
| Standardisation | `StandardScaler` | KNN, SVM, PCA, gradient descent models |
| Normalisation | `MinMaxScaler` | Neural networks, when bounded range needed |
| Label encoding | `LabelEncoder` | Ordinal categories, tree models |
| One-hot encoding | `OneHotEncoder` / `pd.get_dummies` | Nominal categories, linear/distance models |
| Train/test split | `train_test_split` | Always — before any fitting! |

### ⚠️ The golden rule
**Fit preprocessing (scalers, encoders) on the training set only**, then transform both sets. Fitting on the full dataset causes **data leakage** and gives unreliable model evaluation.